In [1]:
import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo
from ray.rllib.models import ModelCatalog
from environment_rllib import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions

In [2]:
import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo
from ray.rllib.models import ModelCatalog
from environment_rllib import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions

# tf1, tf, tfv = try_import_tf()

import tensorflow as tf

In [3]:
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
CONTINUAL = False

In [4]:
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

In [5]:
ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

2021-10-03 05:05:24,239	INFO services.py:1265 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.196',
 'raylet_ip_address': '192.168.0.196',
 'redis_address': '192.168.0.196:6379',
 'object_store_address': 'tcp://127.0.0.1:61328',
 'raylet_socket_name': 'tcp://127.0.0.1:60354',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\Takumi\\AppData\\Local\\Temp\\ray\\session_2021-10-03_05-05-18_724847_4908',
 'metrics_export_port': 60416,
 'node_id': 'df18c57878199278d4b95ef2cb30c8e049d1ea93adbded07bad439d5'}

In [6]:
ModelCatalog.register_custom_model('my_model', DenseNetModelLarge)

# config = {"env": MyEnv,
#           "num_workers": NUM_WORKERS,
#           "num_gpus": NUM_GPUS,
#           "num_cpus_per_worker": NUM_CPUS_PER_WORKER,
#           "num_sgd_iter": NUM_SGD_ITER,
#           "lr": LEARNING_RATE,
#           "gamma": GAMMA,  # default=0.99
#           "model": {"custom_model": "my_model"}
#           # "framework": framework
#           }  # use tensorflow 2
config = {"env": MyEnv,"num_gpus": 0,"num_workers": 1, "num_cpus_per_worker": 0}
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

In [7]:
if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

In [8]:
trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    model_path = PROJECT + '/checkpoints/' + 'fwd_1/checkpoint_000351/checkpoint-351'
    trainer.restore(checkpoint_path=model_path)

2021-10-03 05:05:33,526	INFO trainer.py:714 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-10-03 05:05:33,526	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-10-03 05:05:33,528	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-10-03 05:05:40,080	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [9]:
models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
text_name = models_dir + TRIAL + '.txt'
with open(text_name, "w") as fp:
    trainer.get_policy().model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
png_name = models_dir + TRIAL + '.png'
plot_model(trainer.get_policy().model.base_model, to_file=png_name, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [10]:
# Instanciate the evaluation env
eval_env = MyEnv({})

C:\Users\Takumi\anaconda3\envs\AI_Ray\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [11]:
# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

# Training & evaluation
results_dir = os.path.join('./' + PROJECT + '/results/')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'

In [12]:
for steps in range(10001):
    # Training
    results = trainer.train()

    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n----------------- Evaluation at steps:{steps} starting ! -----------------')
        #print(pretty_print(results))
        check_point = trainer.save(checkpoint_dir=check_point_dir)

        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            obs = eval_env.reset()
            done = False

            while not done:
                action_dict = {}
                for j in range(eval_env.num_red):
                    if eval_env.red.alive[j]:
                        action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])

                obs, rewards, dones, infos = eval_env.step(action_dict)
                done = dones["__all__"]
                print(f'rewards:{rewards}')
                eval_env.render()

        f.close()

ray.shutdown()

RayTaskError(ValueError): [36mray::RolloutWorker.par_iter_next()[39m (pid=21104, ip=192.168.0.196, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000209F3A5AB38>)
  File "python\ray\_raylet.pyx", line 536, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 486, in ray._raylet.execute_task.function_executor
  File "C:\Users\Takumi\anaconda3\envs\AI_Ray\lib\site-packages\ray\_private\function_manager.py", line 563, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\Users\Takumi\anaconda3\envs\AI_Ray\lib\site-packages\ray\util\iter.py", line 1151, in par_iter_next
    return next(self.local_it)
  File "C:\Users\Takumi\anaconda3\envs\AI_Ray\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 346, in gen_rollouts
    yield self.sample()
  File "C:\Users\Takumi\anaconda3\envs\AI_Ray\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 744, in sample
    batches = [self.input_reader.next()]
  File 